In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [2]:
print("Tensorflow version: {}".format(tf.version.VERSION))
print("Eager mode: {}".format(tf.executing_eagerly()))
print("Hub version: {}".format(hub.__version__))
print("GPU is {}".format("available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE"))

Tensorflow version: 2.0.0
Eager mode: True
Hub version: 0.7.0
GPU is NOT AVAILABLE


In [3]:
# 完整的数据按照5:5的比例进行切割得到训练集(train)和测试集(test)
# 将训练集按照 6:4 的比例进行切割得到新的训练集(train)和验证集(validation)
train_validation_split = tfds.Split.TRAIN.subsplit([6, 4])

In [4]:
(train_data, validation_data), test_data = tfds.load(
    name='imdb_reviews', # 网络电影数据库(Internet Movie Database)
    split=(train_validation_split, tfds.Split.TEST),
    as_supervised=True
)

ERROR:absl:Failed to construct dataset imdb_reviews


ProxyError: HTTPSConnectionPool(host='storage.googleapis.com', port=443): Max retries exceeded with url: /tfds-data/?prefix=dataset_info/imdb_reviews/plain_text/0.1.0/ (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None)))

In [5]:
# 查看数据
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch # <shape=(10,), numpy=array(...)>
train_labels_batch # <shape=(10,), numpy=array([1, 1, 1,....0])

NameError: name 'train_data' is not defined

In [6]:
# 构建模型
# 迁移学习来处理预处理文本(预训练文本嵌入 text embedding)

embedding = "https://hub.tensorflow.google.cn/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding,
                          input_shape=[],
                          dtype=tf.string,
                          trainable=True)

In [8]:
hub_layer(train_examples_batch[:3])

NameError: name 'train_examples_batch' is not defined

In [9]:
# 构建完整的模型
model = tf.keras.Sequential([
    # Tensorflow Hub 层
    # 这一层使用一个预训练的保存好的模型来将句子映射为嵌入向量（embedding vector）。
    # 我们所使用的预训练文本嵌入（embedding）模型(google/tf2-preview/gnews-swivel-20dim/1)将句子切割为符号，
    # 嵌入（embed）每个符号然后进行合并。
    # 最终得到的维度是：(num_examples, embedding_dimension)
    hub_layer,
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.binary_crossentropy,
    metrics=['accuracy']
)

In [11]:
# 训练模型
# 以 512 个样本的 mini-batch 大小迭代 20 个 epoch 来训练模型。
# 这是指对 x_train 和 y_train 张量中所有样本的的 20 次迭代。
# 在训练过程中，监测来自验证集的 10,000 个样本上的损失值（loss）和准确率（accuracy）
history = model.fit(
    train_data.shuffle(10000).batch(512),
    epochs=20,
    validation_data=validation_data.batch(512),
    verbose=1
)

NameError: name 'train_data' is not defined

In [ ]:
# 评估模型
results = model.evaluate(test_data.)